In [1]:
# import plotly.plotly as py
import plotly.offline
from plotly.graph_objs import Scatter, Layout
import numpy as np
import pandas as pd

In [2]:
def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx]


def find_nearest_sorted(sorted_array, value):
    idx = np.searchsorted(sorted_array, value, side="left")
    if (idx > 0 and (
        idx == len(sorted_array)
        or abs(value - sorted_array[idx-1]) < abs(value-sorted_array[idx]))):
        return sorted_array[idx-1], idx-1
    else:
        return sorted_array[idx], idx

In [20]:
# target resistances
initial_gaps = np.array( [1.367e-9, 1.5e-9, 1.6e-9, 1.7e-9])
n_gaps = initial_gaps.shape[0]
levels = 1024
target_levels = 32
rs_min = 1e3
r_loads = np.linspace(rs_min, levels*rs_min, levels)
r_index = np.linspace(1, levels, levels)
simple_index = np.linspace(1, levels)

In [21]:
# plotly.tools.set_credentials_file(username='fmu', api_key='NIDiVLcNRVtwNzBnfpLJ')
plotly.tools.set_config_file(world_readable=False,
                             sharing='private')
# plotly.tools.set_config_file(world_readable=True,
#                            sharing='public')

In [22]:
full_data = np.genfromtxt('exported_data/1r_last_r_read.csv', delimiter=',')
# data in X0 Y0, X1, Y1 format, grab all Y values
last_r_read = np.array([full_data[full_data.shape[0]-1, 1::2]])
r_length = int(last_r_read.shape[1]/n_gaps)
last_r_read = last_r_read.reshape(n_gaps, r_length)
print(last_r_read.shape)


(4, 256)


In [23]:
plot_2d = True
if plot_2d:
    data_r = []
    for g in last_r_read:
        data_r.append(
            plotly.graph_objs.Scatter(
                x=r_index,
                y=g,
                mode='lines+markers',
                name='r_read_resistance'
            )
        )
    print(len(data_r))
    layout_r0 = plotly.graph_objs.Layout(
        title='Read Resistances vs Load Resistances',
        xaxis=dict(
            title='Load Resistance [ohm]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Read Resistance [ohm]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        )
    )
    fig_r0 = plotly.graph_objs.Figure(data=data_r, layout=layout_r0)
    plotly.offline.plot(fig_r0, filename = 'read_resistance.html')

4


In [38]:
target_r = np.zeros([n_gaps, target_levels])
for g_idx, g in enumerate(last_r_read):
    target_r[g_idx] = np.linspace(np.min(g), np.max(g), target_levels)
r_loads = np.zeros(target_r.shape)
# find target gaps for each target resistance at v_read
for g_idx, g in enumerate(target_r):
    for t_idx, r in enumerate(g):
        new_r, r_idx = find_nearest_sorted(g, r)
        target_r[g_idx, t_idx] = new_r
        r_loads[g_idx, t_idx] = r_index[r_idx]

In [39]:
if plot_2d:
    data_r = [
        plotly.graph_objs.Scatter(
                x=simple_index,
                y=r_loads[0,:],
                mode='lines+markers',
                name='load_resistance'
        ),
        plotly.graph_objs.Scatter(
            x=simple_index,
            y=target_r[0,:],
            mode='lines+markers',
            name='load_resistance',            
            yaxis='y2'
        )
    ]
    print(len(data_r))
    layout_r0 = plotly.graph_objs.Layout(
        title='Read Resistances vs Load Resistances',
        xaxis=dict(
            title='Level',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis=dict(
            title='Load Resistance [ohm]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            )
        ),
        yaxis2=dict(
            title='Read Resistance [ohm]',
            titlefont=dict(
                family='Courier New, monospace',
                size=18,
                color='#7f7f7f'
            ),
            overlaying='y',
            side='right'
        )
    )
    
    fig_r0 = plotly.graph_objs.Figure(data=data_r, layout=layout_r0)
    plotly.offline.plot(fig_r0, filename = 'load_resistances.html')

2
